### Using Ollama with Python
LiteLLM is a Python library that provides a unified interface to interact with various LLMs, including those run by Ollama.

To use Ollama with LiteLLM, you first need to ensure that your Ollama server is running. Then, you can use the litellm.completion function to make requests to the server. Here's an example of how to do this:


In [1]:
! pip install litellm

   ---------------------------------------- 0.0/6.7 MB ? eta -:--:--
    --------------------------------------- 0.1/6.7 MB 2.2 MB/s eta 0:00:04
   - -------------------------------------- 0.3/6.7 MB 2.6 MB/s eta 0:00:03
   --- ------------------------------------ 0.5/6.7 MB 3.7 MB/s eta 0:00:02
   ----- ---------------------------------- 0.9/6.7 MB 4.6 MB/s eta 0:00:02
   ------- -------------------------------- 1.3/6.7 MB 5.2 MB/s eta 0:00:02
   --------- ------------------------------ 1.6/6.7 MB 5.3 MB/s eta 0:00:01
   ----------- ---------------------------- 1.9/6.7 MB 5.5 MB/s eta 0:00:01
   ------------ --------------------------- 2.2/6.7 MB 5.8 MB/s eta 0:00:01
   --------------- ------------------------ 2.6/6.7 MB 6.1 MB/s eta 0:00:01
   ----------------- ---------------------- 2.9/6.7 MB 6.1 MB/s eta 0:00:01
   ------------------ --------------------- 3.2/6.7 MB 6.1 MB/s eta 0:00:01
   ------------------- -------------------- 3.4/6.7 MB 6.1 MB/s eta 0:00:01
   ----------------

In [2]:
! pip install ollama

In [1]:
from litellm import completion

C:\Users\apt4c\anaconda3\Lib\site-packages\pydantic\_internal\_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


#### Prompt the model

In [2]:
response = completion(
model="ollama/llama2",
messages=[{ "content": "What should I consider when deciding between a sales executive role and a sales enablement role?", "role": "user"}],
api_base="http://localhost:11434"
)

print(response.choices[0].message.content)

Great question! When deciding between a sales executive role and a sales enablement role, there are several factors to consider. Here are some key differences between the two roles that may help you make your decision:

Sales Executive Role:

* Focuses on closing deals and meeting sales targets
* Involves working directly with customers to understand their needs and close sales
* Typically involves a high level of commission-based compensation
* May require a more aggressive and pressure-driven approach to sales

Sales Enablement Role:

* Focuses on providing support and resources to sales teams to help them close deals more effectively
* Involves creating and delivering training programs, content, and tools to help sales teams improve their skills and product knowledge
* Typically involves a different compensation structure, such as salary + bonus or commission-based
* May require a more collaborative and supportive approach to sales

Here are some factors to consider when deciding be

---

#### RAG

Source: https://www.datacamp.com/tutorial/deepseek-r1-ollama

Install dependencies

*gradio* for UI  
*langchain* for tool integration  
*chromadb* for vector database

In [40]:
!pip install -U pydantic pydantic_core 

  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl.metadata (6.7 kB)
Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl (2.0 MB)
  Attempting uninstall: pydantic_core
    Found existing installation: pydantic_core 2.18.4
    Uninstalling pydantic_core-2.18.4:
      Successfully uninstalled pydantic_core-2.18.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.4
    Uninstalling pydantic-2.7.4:
      Successfully uninstalled pydantic-2.7.4


In [3]:
# FAILED
#!pip install chromadb

  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached posthog-3.11.0-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached opentelemetry_api-1.29.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.29.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.50b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.29.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached 

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: Could not build wheels for chroma-hnswlib, which is required to install pyproject.toml-based projects


In [72]:
!pip install langchain 
!pip install gradio 
!pip install -U langchain-community

In [84]:
!pip install faiss-cpu

In [140]:
#pip list

Import modules

In [141]:
import gradio as gr
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
#from langchain_community import embeddings
from langchain.vectorstores.faiss import FAISS
import ollama
import re

#### Process uploaded PDF

In [147]:
def process_pdf(pdf_bytes):
    if pdf_bytes is None:
        return None, None, None
    loader = PyMuPDFLoader(pdf_bytes)
    data = loader.load()
    print('data:', data)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
    chunks = text_splitter.split_documents(data)
    print('chunks[0]:', chunks[0])
    print('chunks[1]:', chunks[1])
    #embed = embeddings.ollama.OllamaEmbeddings(base_url='http://192.168.0.200:11434', model="ollama/llama2")
    #embed = OllamaEmbeddings(base_url='http://localhost:11434', model="ollama/llama2")
    embed = OllamaEmbeddings(base_url='http://localhost:11434', model="llama2:latest")
    print('embed:', embed)
    #vectorstore=Chroma.from_documents(documents=chunks, embedding=embeddings)

    # Create the FAISS index for document retrieval
    vectorstore = FAISS.from_documents(chunks, embed)
    
    retriever = vectorstore.as_retriever()
    print('vectorstore:', vectorstore)
    
    return text_splitter, vectorstore, retriever

In [148]:
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [149]:
def ollama_llm(question, context):
   formatted_prompt = f"Question: {question}\n\nContext: {context}"
   response = ollama.chat(model="llama2:latest", messages=[{'role': 'user', 'content': formatted_prompt}])
   print('response:',response)
   response_content = response['message']['content']
    
   # Remove content between <think> and </think> tags to remove thinking output
   final_answer = re.sub(r'<think>.*?</think>', '', response_content, flags=re.DOTALL).strip()
   print('final_answer:',final_answer)
    
   return final_answer

In [150]:
def rag_chain(question, text_splitter, vectorstore, retriever):
    retrieved_docs = retriever.invoke(question)
    formatted_content = combine_docs(retrieved_docs)
    return ollama_llm(question, formatted_content)

In [ ]:
def ask_question(pdf_bytes, question):
    text_splitter, vectorstore, retriever = process_pdf(pdf_bytes)
    if text_splitter is None:
        return None  # No PDF uploaded
    result = rag_chain(question, text_splitter, vectorstore, retriever)
    return {result}
    
interface = gr.Interface(
    fn=ask_question,
    inputs=[gr.File(label="Upload PDF (optional)"), gr.Textbox(label="Ask a question")],
    outputs="text",
    title="Ask questions about your PDF",
    description="Use llama2 to answer your questions about the uploaded PDF document.",
)
interface.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


data: [Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2025-01-29T12:17:12-05:00', 'source': 'C:\\Users\\apt4c\\AppData\\Local\\Temp\\gradio\\34b58d949817ee04404996ee6afb28921aed932ffc8e59c68771a4131f0f4c24\\CV_APT_latest.pdf', 'file_path': 'C:\\Users\\apt4c\\AppData\\Local\\Temp\\gradio\\34b58d949817ee04404996ee6afb28921aed932ffc8e59c68771a4131f0f4c24\\CV_APT_latest.pdf', 'total_pages': 8, 'format': 'PDF 1.7', 'title': 'CV_APT_latest.doc', 'author': 'Adam Tashman', 'subject': '', 'keywords': '', 'moddate': '2025-01-29T12:17:12-05:00', 'trapped': '', 'page': 0}, page_content='Adam P. Tashman, PhD, PRM\nSUMMARY\n\uf0b7Data science leader, educator, author\n\uf0b7Experienced in leading and building teams of data scienƟsts and quanƟtaƟve researchers\n\uf0b7QuanƟtaƟve analyst/researcher with extensive experience in modeling, predicƟon, programming, risk\n\uf0b7Specialize in machine learning, data mining, stochasƟc processes, risk models, Ɵme series